In [ ]:
# rlc_circuit.ipynb
# Cell 1

import numpy as np
import turtle as t
import matplotlib.pyplot as plt


def f(x, r, l, c, i_init): #define analytic solution
    b = r / (2 * l) #beta
    w = np.sqrt(1 / (l * c) - (r**2 / 4 * l**2)) #omega
    c2 = (i_init * b) / w #constant c2
    c1 = i_init #constant c1
    return np.exp(-b * x) * (c1 * np.cos(w * x) + c2 * np.sin(w * x))

def plot(anim):
    x = np.linspace(0, 1, 5_000) #[0,1] chopped into 5,000 intervals
    
    # set constants
    l = c = 0.01 #inductance and capacitance (henries and farads respectively)
    r = 0.1 #resistantce, ohms

    # Set initial condition, initial current
    i_init = 1
    
    for i in range(len(x)): #iteratively draw points
        anim.goto(x[i] * 400, f(x[i], r, l, c, i_init) * 400)

def main():
    plt.close("all")
    anim = t.Turtle()
    anim.color("red")
    anim.speed("fastest")
    plot(anim)
    
    plt.show()
    
    
main()

In [ ]:
# rlc_circuit.ipynb
# Cell 2

%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import AutoMinorLocator
from scipy.integrate import solve_ivp


def f(x, r, l, c, i_init):
    b = r / (2 * l)
    w = np.sqrt(1 / (l * c) - (r**2 / 4 * l**2))
    c2 = (i_init * b) / w
    c1 = i_init
    return np.exp(-b * x) * (c1 * np.cos(w * x) + c2 * np.sin(w * x))

def model(time, state_vector, phase_constant, damping_constant):
    theta, i = state_vector  # unpack dependent variables
    #I'm calling the first derivative of i "theta"
    d_theta = -damping_constant * theta - phase_constant * i
    d_i = theta
    return d_theta, d_i


def plot(ax):
    x = np.linspace(0, 1, 10_000) #[0,1] chopped into 10,000 intervals
    # Precalculate phase constant
    l = c = 0.01 #(henries and farads respectively)
    phase_constant = 1 / (l * c)

    # Set damping_constants
    r = 0.1 #ohms
    underdamped_constant = r / l

    # Set initial conditions
    theta_initial = 0 #no change in I(t) to start
    i_init = 1

    # Set model duration (seconds)
    time_initial = 0
    time_final = 1

    # Calculate for an underdamped wave
    sol = solve_ivp(
        model,
        (time_initial, time_final),
        [theta_initial, i_init],
        max_step=0.001, #curve looks "choppy" if the max_step is too large
        args=[phase_constant, underdamped_constant],
    )
    time_steps = sol.t
    theta_underdamped = sol.y[1]

    ax.plot(
        time_steps,
        theta_underdamped,
        label="SciPy RK45", #numeric in blue
        color="blue",
        linestyle="dashed",
        zorder=3,
        linewidth=5
    )
    
    ax.plot(
        x,
        f(x, r, l, c, i_init),
        label=r"$I(t) = e^{- \beta t}(I_0 \cos{\omega t}$"
        r"$ + \frac{I_{0} \beta}{\omega} \sin{\omega t})$", #analytic in red
        color="red",
        linestyle="solid",
        zorder=2,
    )

    ax.set_title(r"$\frac{d^{2}I(t)}{dt^2} =-\frac{R}{L}\frac{dI(t)}{dt}$"
                 r'$ \ - \frac{I(t)}{LC}$', fontsize=20)
    ax.set_xlabel(r"$\mathrm{Time} \ t \ \mathrm{(sec)}$", fontsize=16)
    ax.set_ylabel(r"$ \mathrm{Current} \ I(t) \ \mathrm{(Amp)}$", fontsize=16)
    ax.axhline(y=0.0, color="lightgray")
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.legend(loc="upper right", fontsize=14)

def main():
    plt.close("all")
    fig = plt.figure(" ")
    fig.set_size_inches(9, 6)
    gs = fig.add_gridspec(1, 1)
    ax = fig.add_subplot(gs[0, 0])
    plot(ax)
    
    plt.show()
    
main()